In [ ]:
from bindsnet.network import load_network
from bindsnet.encoding import poisson
from bindsnet.analysis.plotting import plot_weights
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pickle
import torch
from encoder import *
import time as T
plt.rcParams["figure.figsize"] = (20, 20)
lfname = 'logs/' + str(int(T.time())) + '_anomaly.txt'

In [ ]:
track2 = pickle.load(open('encoding/test_anomaly.p', 'rb'))

In [ ]:
network = load_network('trained.net')
network.connections[('X', 'Ae')].update_rule = None
exc_monitor = network.monitors['exc']

In [ ]:
time = 500

In [ ]:
def log(msg):
    print(msg, end='')
    with open(lfname, 'a') as f:
        f.write(msg)

In [ ]:
def detect(spikes):
    cols = torch.sum(spikes, dim=1)
    for i, v in enumerate(cols):
        if v != 0:
            log(f'Neuron: {i} Spikes: {int(v)}\n')

In [ ]:
for i in range(1, len(track2)):
    orig = torch.cat((track2[i-1], track2[i]), dim=1)
    print(orig.shape)
    pt = orig.view(-1)
    pt = poisson(pt, time)
    
    inpts = {'X': pt}
    network.run(inpts=inpts, time=time)
    spikes = exc_monitor.get('s')
    
    log(f'Iteration {i}\n')
    detect(spikes)
    
    fig = plt.figure(figsize=(20, 20))
    plt.subplot(2, 2, 1)
    plt.imshow(spikes, cmap='binary')
    plt.subplot(2, 2, 2)
    plt.imshow(orig, cmap='gist_gray')
    plt.show()
    
    clear_output(wait=True)

In [ ]:
network.connections[('X', 'Ae')].w.shape

In [ ]:
plot_weights(network.connections[('X', 'Ae')].w)